In [ ]:
from openai import OpenAI
from tqdm import tqdm
import json
import os

In [5]:
def gpt4o_forecast_bitcoin_price(news_json, lookback_values, prediction_window):
    system_prompt = (
        "You are an expert financial forecaster specializing in Bitcoin. "
        "Given recent Bitcoin-related news (as a JSON list) and the last N days of Bitcoin prices, "
        "your job is to predict the next M day(s) of Bitcoin price. "
        "Respond ONLY with a comma-separated list of the forecasted Bitcoin prices."
    )
    user_prompt = (
        f"News (JSON): {news_json}\n"
        f"Recent Bitcoin prices: {lookback_values}\n"
        f"Forecast the next {prediction_window} day(s) of Bitcoin price as a comma-separated list."
    )
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=32,
        temperature=0.0,
    )
    forecast_text = response.choices[0].message.content.strip()
    try:
        forecast = [float(x) for x in forecast_text.split(",")]
    except Exception:
        forecast = [float('nan')] * prediction_window
    return forecast

In [ ]:
def run_gpt4o_baseline(val_loader, prediction_window, news_col='summary'):
    all_preds = []
    all_targets = []
    for batch in tqdm(val_loader):
        texts = batch['text']  # list of lists of news summaries (per lookback day)
        values = batch['value']  # tensor: (batch, lookback, features)
        targets = batch['target']  # tensor: (batch, prediction_window, features)
        for i in range(len(texts)):
            news_json_list = []
            for summary in texts[i]:
                news_json_list.append({
                    "summary": summary
                })
            news_json = json.dumps(news_json_list, ensure_ascii=False)
            lookback_values = values[i, :, 0].tolist()  # assuming 1D value
            forecast = gpt4o_forecast_bitcoin_price(news_json, lookback_values, prediction_window)
            all_preds.append(forecast)
            all_targets.append(targets[i, :, 0].tolist())
    return all_preds, all_targets